In [17]:
# Create a function that extracts price data from yahoo using a ticker code
def get_yahoo_data(ticker_and_isin: List[str]) -> Optional[pd.DataFrame]:
    yahoo_code = ticker_and_isin[0]
    stock_isin = ticker_and_isin[1]
    try:
        ticker = yf.Ticker(yahoo_code)
    except:
        ticker = None
    # Get some basic stock info
    try:
        stock_info = ticker.info
    except:
        stock_info = None
    try: 
        yahoo_sector = stock_info['sector']
    except:
        yahoo_sector = None
    try:
        yahoo_industry = stock_info['industry']
    except:
        yahoo_industry = None
    try:
        yahoo_full_time_employees = stock_info['fullTimeEmployees']
    except:
        yahoo_full_time_employees = None
    try:
        yahoo_shares_outstanding = stock_info['sharesOutstanding']
    except:
        yahoo_shares_outstanding = None
    if yahoo_sector or yahoo_industry or yahoo_full_time_employees or yahoo_shares_outstanding:
        stock_info_dict = {
            'stock_isin' : stock_isin,
            'yahoo_sector': [yahoo_sector],
            'yahoo_industry': [yahoo_industry],
            'yahoo_full_time_employees': [yahoo_full_time_employees],
            'yahoo_shares_outstanding': [yahoo_shares_outstanding]
        }
        stock_info = pd.DataFrame.from_dict(stock_info_dict)
    else:
        stock_info = pd.DataFrame()
    try:
        # Get the return history
        prices = ticker.history(period="5y")
        renaming_dict = {
            'Date': 'date',
            'Open': 'open_price',
            'High': 'high_price',
            'Low': 'low_price',
            'Close': 'close_price',
            'Adj Close': 'adjusted_close_price',
            'Volume': 'trading_volume',
            'Dividends': 'dividends',
            'Stock Splits': 'stock_splits',
        }
        prices.reset_index(inplace=True)
        prices.rename(columns=renaming_dict, inplace=True)
        prices['stock_isin'] = stock_isin
    except:
        prices = pd.DataFrame()
    # Get financial statement history
    try:
        financials = ticker.financials
        financials.index.set_names(['financial_statement_measure'], inplace=True)
        financials.reset_index(inplace=True)
        financials = pd.melt(financials, id_vars='financial_statement_measure',
                                    var_name='date', value_name='amount')
        financials['stock_isin'] = stock_isin
    except:
        financials = pd.DataFrame()
    # Get earnings history
    try:
        earnings = ticker.earnings
        earnings.index.set_names(['year'], inplace=True)
        earnings.reset_index(inplace=True)
        earnings = pd.melt(earnings, id_vars='year',
                                        var_name='earnings_measure', value_name='amount')
        earnings['stock_isin'] = stock_isin
    except:
        earnings = pd.DataFrame()
    # Get balance sheet history
    try:
        balance_sheet = ticker.balance_sheet
        balance_sheet.index.set_names(['balance_sheet_measure'], inplace=True)
        balance_sheet.reset_index(inplace=True)
        balance_sheet = pd.melt(balance_sheet, id_vars='balance_sheet_measure',
                                        var_name='date', value_name='amount')
        balance_sheet['stock_isin'] = stock_isin
    except:
        balance_sheet = pd.DataFrame()
    # Get cashflow history
    try:
        cashflow = ticker.cashflow
        cashflow.index.set_names(['cash_flow_measure'], inplace=True)
        cashflow.reset_index(inplace=True)
        cashflow = pd.melt(cashflow, id_vars='cash_flow_measure',
                                        var_name='date', value_name='amount')
        cashflow['stock_isin'] = stock_isin
    except:
        cashflow = pd.DataFrame()
    return stock_info, prices, financials, earnings, balance_sheet, cashflow

In [18]:
# Test by extracting Apple stock returns
with no_ssl_verification():
    stock_info, prices, financials, earnings, balance_sheet, cashflow = get_yahoo_data(
        ['AAPL', 'US0378331005'])
print('Stock info:')
print(stock_info.head())
print('----------------------------------------------------------')
print('Historical prices:')
print(prices.head())
print('----------------------------------------------------------')
print('Historical financials:')
print(financials.head())
print('----------------------------------------------------------')
print('Historical earnings:')
print(earnings.head())
print('----------------------------------------------------------')
print('Historical balance sheet:')
print(balance_sheet.head())
print('----------------------------------------------------------')
print('Historical cashflow:')
print(cashflow.head())
print('----------------------------------------------------------')

Stock info:
     stock_isin yahoo_sector        yahoo_industry  yahoo_full_time_employees  yahoo_shares_outstanding
0  US0378331005   Technology  Consumer Electronics                     100000               16319399936
----------------------------------------------------------
Historical prices:
        date  open_price  high_price  low_price  close_price  trading_volume  dividends  stock_splits    stock_isin
0 2017-02-23      32.427      32.451     32.172       32.227        83152800      0.000         0.000  US0378331005
1 2017-02-24      32.080      32.257     31.931       32.257        87106400      0.000         0.000  US0378331005
2 2017-02-27      32.371      32.441     32.168       32.321        81029600      0.000         0.000  US0378331005
3 2017-02-28      32.356      32.441     32.267       32.335        93931600      0.000         0.000  US0378331005
4 2017-03-01      32.548      33.081     32.479       32.996       145658400      0.000         0.000  US0378331005
------

In [19]:
# Prepare list of ticker information to pass into our iterator function.
relevant_tickers = stock_info_euronext[[
    'yahoo_code', 'stock_isin']].dropna()
tickers = relevant_tickers.to_numpy().tolist()
len(tickers)

1687

In [20]:
# Define function to run through all the ticker codes and extract price data
# https://docs.python.org/3/library/concurrent.futures.html
def iterate_tickers(tickers: List[List[str]])-> List[pd.DataFrame]:
    stock_info_list, prices_list, financials_list, earnings_list, balance_sheet_list, cashflow_list = (list() for i in range(6))
    no_of_tickers = len(tickers)
    logging.info(f'Starting extraction of data for {no_of_tickers} tickers')
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        # Start the load operations and mark each future with its ticker and isn number
        download_ticker_info = {executor.submit(
            get_yahoo_data, ticker_and_isn): ticker_and_isn for ticker_and_isn in tickers}
        for future in concurrent.futures.as_completed(download_ticker_info):
            current_download = download_ticker_info[future]
            stock_info, prices, financials, earnings, balance_sheet, cashflow = future.result()
            stock_info_list.append(stock_info)
            prices_list.append(prices)
            financials_list.append(financials)
            earnings_list.append(earnings)
            balance_sheet_list.append(balance_sheet)
            cashflow_list.append(cashflow)
            # Check mark \u2713 cross mark \u2717
            # Retrieval status
            SI = '\u2717' if stock_info.empty else '\u2713'
            F = '\u2717' if financials.empty else '\u2713'
            E = '\u2717' if earnings.empty else '\u2713'
            BS = '\u2717' if balance_sheet.empty else '\u2713'
            CF = '\u2717' if cashflow.empty else '\u2713'
            logging.info(f'{current_download}: SI{SI},F{F},E{E},BS{BS},CF{CF}')
    return stock_info_list, prices_list, financials_list, earnings_list, balance_sheet_list, cashflow_list


In [21]:
# Loop through all tickers to acquire data
with no_ssl_verification():
  stock_info_list, price_list, financials_list, earnings_list, balance_sheet_list, cashflow_list = iterate_tickers(
      tickers)

2022-02-23 05:08:11 PM: Starting extraction of data for 1687 tickers
2022-02-23 05:08:16 PM: ['2020.OL', 'BMG9156K1018']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 05:08:16 PM: ['2MXBS.PA', 'FR0014000TB2']: SI✗,F✗,E✗,BS✗,CF✗
2022-02-23 05:08:17 PM: ['2MX.PA', 'FR0014000T90']: SI✓,F✗,E✗,BS✗,CF✗
2022-02-23 05:08:17 PM: ['2CRSI.PA', 'FR0013341781']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 05:08:18 PM: ['ALMIL.PA', 'FR0010285965']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 05:08:21 PM: ['5PG.OL', 'DK0060945467']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 05:08:23 PM: ['ASP.PA', 'FR0000076887']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 05:08:23 PM: ['AALB.AS', 'NL0000852564']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 05:08:27 PM: ['MLATV.PA', 'FR0010050773']: SI✗,F✗,E✗,BS✗,CF✗


- MLATV.PA: No data found, symbol may be delisted


2022-02-23 05:08:27 PM: ['ABI.BR', 'BE0974293251']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 05:08:28 PM: ['AB.PA', 'FR0010557264']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 05:08:28 PM: ['AASB.OL', 'NO0010672181']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 05:08:28 PM: ['ABCA.PA', 'FR0004040608']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 05:08:33 PM: ['ABG.OL', 'NO0003021909']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 05:08:33 PM: ['ABEO.PA', 'FR0013185857']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 05:08:34 PM: ['ABN.AS', 'NL0011540547']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 05:08:34 PM: ['ABNX.PA', 'FR0012616852']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 05:08:35 PM: ['ABVX.PA', 'FR0012333284']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 05:08:38 PM: ['ABO.BR', 'BE0974278104']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 05:08:38 PM: ['ACPH.BR', 'GB00BYWF9Y76']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 05:08:39 PM: ['ACCEL.AS', 'NL0009767532']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 05:08:39 PM: ['ACAN.PA', 'FR0000064602']: SI✓,F✓,E✗,BS✓,CF✓
2022-02-23 05:08:40 PM: ['ACCB.BR', 'BE0003696102']: SI✓,F✓,E✗,BS✓,CF✓
2022-02-23 

- CUR.LS: No data found, symbol may be delisted


2022-02-23 05:09:54 PM: ['AF.PA', 'FR0000031122']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 05:09:54 PM: ['AD.AS', 'NL0011794037']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 05:09:54 PM: ['A5G.IR', 'IE00BF0L3536']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 05:09:55 PM: ['AI.PA', 'FR0000120073']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 05:10:03 PM: ['MLAIM.PA', 'FR0013285103']: SI✓,F✓,E✗,BS✓,CF✓
2022-02-23 05:10:06 PM: ['AIRX.OL', 'NO0010895568']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 05:10:07 PM: ['ALAIR.PA', 'IT0005450819']: SI✓,F✗,E✗,BS✗,CF✗
2022-02-23 05:10:07 PM: ['AIR.PA', 'NL0000235190']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 05:10:08 PM: ['MLAIR.PA', 'FR0014003V77']: SI✓,F✗,E✗,BS✗,CF✗
2022-02-23 05:10:17 PM: ['AJAX.AS', 'NL0000018034']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 05:10:20 PM: ['AKER.OL', 'NO0010234552']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 05:10:20 PM: ['AKAST.OL', 'NO0010215684']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 05:10:21 PM: ['AKBM.OL', 'NO0010886625']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 05:10:21 PM: ['AKRBP.OL', 'NO0010345853']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-2

...

2022-02-23 06:11:14 PM: ['VTR.PA', 'FR0010309096']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 06:11:21 PM: ['VVY.AS', 'NL00150002Q7']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 06:11:21 PM: ['VIV.PA', 'FR0000127771']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 06:11:24 PM: ['ALVGO.PA', 'FR0011532225']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 06:11:25 PM: ['VWA.BR', 'DE0007664005']: SI✓,F✓,E✗,BS✓,CF✓
2022-02-23 06:11:27 PM: ['VWAP.BR', 'DE0007664039']: SI✓,F✓,E✗,BS✓,CF✓
2022-02-23 06:11:33 PM: ['VLTSA.PA', 'FR0011995588']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 06:11:33 PM: ['VTA.AS', 'GG00B1GHHH78']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 06:11:35 PM: ['VOLUE.OL', 'NO0010894603']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 06:11:38 PM: ['VPK.AS', 'NL0009432491']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 06:11:38 PM: ['VVL.OL', 'NO0003025009']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 06:11:45 PM: ['VGM.OL', 'NO0011037483']: SI✓,F✗,E✗,BS✗,CF✗
2022-02-23 06:11:45 PM: ['VOW.OL', 'NO0010708068']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 06:11:47 PM: ['ALVDM.PA', 'FR0004045847']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 

- MLWEL.PA: No data found for this date range, symbol may be delisted


2022-02-23 06:12:21 PM: ['MF.PA', 'FR0000121204']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 06:12:22 PM: ['WHA.AS', 'NL0000289213']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 06:12:24 PM: ['WEHB.BR', 'BE0003724383']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 06:12:26 PM: ['WEST.OL', 'NO0010768096']: SI✓,F✓,E✗,BS✓,CF✓
2022-02-23 06:12:29 PM: ['MLWEY.PA', 'FR0010768770']: SI✓,F✗,E✗,BS✗,CF✗
2022-02-23 06:12:32 PM: ['MLWRC.PA', 'ES0105399002']: SI✓,F✓,E✗,BS✓,CF✓
2022-02-23 06:12:33 PM: ['ROCK.BR', 'BE0974401334']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 06:12:36 PM: ['WWI.OL', 'NO0010571698']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 06:12:37 PM: ['WWIB.OL', 'NO0010576010']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 06:12:41 PM: ['WILS.OL', 'NO0010252356']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 06:12:43 PM: ['ALWF.PA', 'FR0014000P11']: SI✓,F✗,E✗,BS✗,CF✗
2022-02-23 06:12:44 PM: ['ALWIT.PA', 'FR0013143872']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 06:12:47 PM: ['MLWIZ.PA', 'FR00140047H7']: SI✓,F✗,E✗,BS✗,CF✗
2022-02-23 06:12:48 PM: ['WOLE.BR', 'BE0003600112']: SI✓,F✓,E✗,BS✓,CF✓
2022-0

- CV.PA: No data found, symbol may be delisted


2022-02-23 06:13:23 PM: ['MLZAM.PA', 'ZM0000000037']: SI✓,F✓,E✗,BS✓,CF✓
2022-02-23 06:13:28 PM: ['ZENA.OL', 'CA98936C1068']: SI✓,F✓,E✗,BS✓,CF✓
2022-02-23 06:13:28 PM: ['ZWIPE.OL', 'NO0010721277']: SI✓,F✓,E✓,BS✓,CF✓
2022-02-23 06:13:29 PM: ['ZEN.BR', 'BE0003809267']: SI✓,F✓,E✗,BS✓,CF✓
2022-02-23 06:13:30 PM: ['ORN.OL', 'NO0010934748']: SI✓,F✓,E✓,BS✓,CF✓


In [22]:
# Concat lists with dataframes if need be and save all files to csv
# Euronext stock info
stock_info_euronext.to_csv(
    'raw_data/stock_info_euronext.csv', sep='|', index=False)
# Yahoo stock info
stock_info_yahoo = pd.concat(stock_info_list)
stock_info_yahoo.to_csv(
    'raw_data/stock_info_yahoo.csv', sep='|', index=False)
# Price history
price_data = pd.concat(price_list)
price_data.to_csv(
    'raw_data/price_yahoo.csv', sep='|', index=False)
# Financials history
financials_data = pd.concat(financials_list)
financials_data.to_csv(
    'raw_data/financials_yahoo.csv', sep='|', index=False)
# Earnings history
earnings_data = pd.concat(earnings_list)
earnings_data.to_csv(
    'raw_data/earnings_yahoo.csv', sep='|', index=False)
# Balance sheet history
balance_sheet__data = pd.concat(balance_sheet_list)
balance_sheet__data.to_csv(
    'raw_data/balance_sheet_yahoo.csv', sep='|', index=False)
# Cash flow history
cashflow_data = pd.concat(cashflow_list)
cashflow_data.to_csv(
    'raw_data/cashflow_yahoo.csv', sep='|', index=False)